### 1. Import packages

In [ ]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
else:
    device = torch.device("cpu")   # Use CPU# use cpu run
print(device)
import gym

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

### 2. Helper functions

In [ ]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    return state

### 3. Initialize the environment

In [ ]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': True,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = 25
#ram_dimnetion = 9 if partial
nb_actions = 25
token_size = 27
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)

### 4. Define the network

In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer, BertTokenizerFast
from torch import nn

model_name = 'distilbert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name, use_fast=True)
distilbert_model = DistilBertModel.from_pretrained(model_name)

total_params = sum(p.numel() for p in distilbert_model.parameters())
print(f"Total parameters: {total_params}")

class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, fc1_units=512, fc2_units=256, fc3_units=256):
        super(QNetwork, self).__init__()
        self.distilbert = distilbert_model
        self.fc1 = nn.Linear(distilbert_model.config.hidden_size * state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        # self.fc3 = nn.Linear(fc2_units, fc3_units)
        # self.fc4 = nn.Linear(fc3_units, action_size)
        # 冻结 DistilBERT 的参数
        # for param in self.distilbert.parameters():
        #     param.requires_grad = False


    def forward(self, input_ids, attention_mask):
        # with torch.no_grad():  # 禁用梯度计算以避免冻结的参数被更新
        distilbert_out = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        # Extract the last hidden state or pooled output
        last_hidden_state = distilbert_out.last_hidden_state
        reshaped_hidden_states = last_hidden_state.view(last_hidden_state.shape[0], -1)

        # Pass through fully connected layers
        fc1_out = F.relu(self.fc1(reshaped_hidden_states))
        fc2_out = F.relu(self.fc2(fc1_out))
        # fc3_out = F.relu(self.fc3(fc2_out))
        return self.fc3(fc2_out)


In [ ]:
tokenizer.is_fast

### 5. Define the Replay Buffer

In [ ]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            how many samples stored in buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        # store [s,a,r,s',done] for each sample in buffer
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        states = np.vstack([e.state for e in experiences if e is not None])
        # vstack is add rows together as a single matrix
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = np.vstack([e.next_state for e in experiences if e is not None])
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        #get all the states, actions, rewards, next_states, dones for all elements in this sample batch, each as a single matrix
        #device here is cpu?
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [ ]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 512        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 8                 # for update of target network parameters
LR = 1e-5               # learning rate 
betas=(0.9, 0.999)
weight_decay=0.001
UPDATE_EVERY = 16       # how often to update the network

In [ ]:
def array2str(state):
    state_str = ""
    for i, num in enumerate(state):
        if i == 0:
            state_str += str(round(num / 40)) + " "
        elif i == 4:
            state_str += str(round(num / 100)) + " "
        elif i == 7:
            state_str += str(round(num / 10)) + " "
        elif i == 20:
            state_str += str(round(num / 100)) + " " # 250
        elif i == 21:
            state_str += str(round(num / 6)) + " " # 12
        elif i == 23:
            state_str += str(round(num)) + " " # 10
        elif i >= 9 and i <= 17:
            state_str += str(round(num * 1000)) + " "
        elif i == 18:
            state_str += str(round(num * 100)) + " "
        else:
            state_str += str(round(num)) + " "
        
    return state_str

In [ ]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size

        # Q-Network
        self.qnetwork_local = QNetwork(token_size, action_size).to(device)
        self.qnetwork_target = QNetwork(token_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR, betas=betas)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        # add current (s,r,a,s') and sample a batch and learn if update_every
        self.memory.add(state, action, reward, next_state, done)
        if done:
            self.memory.add(state, action, reward, next_state, done)
            self.memory.add(state, action, reward, next_state, done)
            self.memory.add(state, action, reward, next_state, done)
            self.memory.add(state, action, reward, next_state, done)
            self.memory.add(state, action, reward, next_state, done)
            self.memory.add(state, action, reward, next_state, done)
            self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """        
        state_str = array2str(state)   
        token = tokenizer(state_str, add_special_tokens=True, max_length=token_size, truncation=True, padding='max_length', return_tensors='pt')
        input_ids = token["input_ids"].to(device)
        attention_mask = token["attention_mask"].to(device)
        
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(input_ids, attention_mask)
            
        self.qnetwork_local.train()
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        state_str_list = []
        for state in states:
            state_str = array2str(state)
            state_str_list.append(state_str)

        token = tokenizer(state_str_list, add_special_tokens=True, max_length=token_size, truncation=True, padding='max_length', return_tensors='pt')
        input_ids_batch = token['input_ids'].to(device)
        attention_mask_batch = token['attention_mask'].to(device)

        next_state_str_list = []
        for next_state in next_states:
            next_state_str = array2str(next_state)
            next_state_str_list.append(next_state_str)

        next_token = tokenizer(next_state_str_list, add_special_tokens=True, max_length=token_size, truncation=True, padding='max_length', return_tensors='pt')
        next_input_ids_batch = next_token["input_ids"].to(device)
        next_attention_mask_batch = next_token["attention_mask"].to(device)

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_input_ids_batch, next_attention_mask_batch).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        Q_expected = self.qnetwork_local(input_ids_batch, attention_mask_batch).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        # after this, the parameter of local network will change based on gradient descent
        for param in self.qnetwork_local.parameters():
            if param.grad is not None:
                param.grad.data.clamp_(-1, 1)
        # stabilize traning to keep grad between (-1,1)
        self.optimizer.step()

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())
    def save(self,name):
        torch.save(self.qnetwork_local.state_dict(),'./checkpoint/model'+name+'.pth')
        print('saved')
        return

In [ ]:
def get_reward(state, n_action, w_action, next_state, done, k1, k2, k3, k4):
    if done:
        reward = k1*state[4] - k2*n_action - k3*w_action
        return reward
    else:
        reward = -k2*n_action - k3*w_action 
        return reward


### DQN for nitrogen management

In [ ]:
agent = Agent(state_size=ram_dimensions, action_size=nb_actions)

In [ ]:
import time
import numpy as np
from collections import deque

def dqn(n_episodes=2000, max_t=500, eps_start=1.0, eps_end=0, eps_decay=0.994, solved=8000, optimize=True, exp=0):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    start = time.time()                # Start time
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    list_eps = []
    eps = eps_start                    # initialize epsilon
    n_amount = 0
    w_amount = 0
    yield_amount = 0
    n_amount_list = []
    n_amount_window = deque(maxlen=100)
    w_amount_list = []
    w_amount_window = deque(maxlen=100)
    leaching_sum_list = []
    leaching_sum_window = deque(maxlen=100)
    yield_list = []
    yield_window = deque(maxlen=100)
    y = 0
    # Update training policy here by updating the weights below
    k1 = 0.158
    k2 = 0.79
    k3 = 1.1
    k4 = 0
    
    for i_episode in range(1, n_episodes + 1):

        time1 = time.time()
        
        # Reset env and score at the beginning of episode
        state = env.reset() # reset the environment and get current state
        state = dict2array(state)
        score = 0 # initialize the score
        n_amount = 0
        w_amount = 0
        leaching_sum = 0
        
        for t in range(max_t):
            action1 = agent.act(state, eps) if optimize else 12
            action = {
                'anfer': (action1 % 5) * 40,  # if mode == fertilization or mode == all; nitrogen to fertilize in kg/ha
                'amir': int(action1 / 5) * 6,  # if mode == irrigation or mode == all; water to irrigate in L/ha
            }
            if state[0] >= 10000:
                action['anfer'] = 0
            if state[21] >= 1600:
                action['amir'] = 0
            
            next_state, reward, done, _ = env.step(action)  # send the action to the environment
            
            if done:
                y = state[4]
                yield_amount = y
                next_state = state
                reward = get_reward(state, action['anfer'], action['amir'], next_state, done, k1, k2, k3, k4)
                agent.step(state, action1, reward, next_state, done)
                score += reward
                n_amount_list.append(n_amount)
                w_amount_list.append(w_amount)
                n_amount_window.append(n_amount)
                w_amount_window.append(w_amount)
                leaching_sum_window.append(leaching_sum)
                yield_list.append(y)
                yield_window.append(y)
                break
            
            n_amount += action['anfer']
            w_amount += action['amir']
            next_state = dict2array(next_state)
            reward = get_reward(state, action['anfer'], action['amir'], next_state, done, k1, k2, k3, k4)
            agent.step(state, action1, reward, next_state, done)
            state = next_state
            score += reward
        
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        
        if score > 1400 and yield_amount > 11000:
            agent.save(str(i_episode))
        
        eps = max(eps_end, eps_decay * eps)  # decrease epsilon
        list_eps.append(eps)
        
        if i_episode % 1 == 0:
            print('\rEpisode {}/{} \t Score: {:.2f}'.format(i_episode, n_episodes, score))
            print('Epsilon: {}'.format(eps))
            print('Nitrogen amount is', n_amount)
            print('Irrigation amount is', w_amount)
            print('Yield amount is', yield_amount)
            
        if np.mean(scores_window) > solved:
            print('Game Solved after {} episodes'.format(i_episode))
            break

        time2 = time.time()
        print("one epoch time:", time2 - time1)
    
    time_elapsed = time.time() - start
    print("Time Elapse: {:.2f} seconds")
    
    return scores, list_eps, n_amount_list, w_amount_list, yield_list

In [ ]:
scores, list_eps, n_amount_list, w_amount_list, yield_list = dqn(n_episodes=3000,exp=1)

In [ ]:
# Create a figure and axes for subplots
fig, axs = plt.subplots(3, 2)

# Plot each dataset on its own subplot
axs[0, 0].plot(scores)
axs[0, 0].set_title('Scores')

axs[0, 1].plot(list_eps)
axs[0, 1].set_title('List Eps')

axs[1, 0].plot(n_amount_list)
axs[1, 0].set_title('N Amount List')

axs[1, 1].plot(w_amount_list)
axs[1, 1].set_title('W Amount List')

axs[2, 0].plot(yield_list)
axs[2, 0].set_title('yield Amount List')

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the plot as a PDF file
plt.savefig('combined_plots_distilbert.pdf')

# Show the plot
plt.show()


In [ ]:
print("Length of scores:", len(scores))
print("Length of list_eps:", len(list_eps))
print("Length of n_amount_list:", len(n_amount_list))
print("Length of w_amount_list:", len(w_amount_list))
print("Length of yield_list:", len(yield_list))

In [ ]:
df = pd.DataFrame({
    'Scores': scores,
    'List_eps': list_eps,
    'N_amount_list': n_amount_list,
    'W_amount_list': w_amount_list,
    'Yield_list': yield_list
})

# save DataFrame to Excel file
df.to_excel('data_distilbert.xlsx', index=False)